In [5]:
# 实时翻译机器人
import os
from openai import OpenAI

os.environ['OPENAI_API_KEY'] = '${YOUR_API_KEY}'
client = OpenAI()

# 1. 语音识别成文字
audio_file = open("input.m4a", "rb")
cn_text = client.audio.transcriptions.create(
    model="whisper-1",
    file=audio_file,
    response_format="json",
    # language="zh",
    temperature=0,
    # prompt="",
)
print(f"origin voice: {cn_text}")

print(f"type of cn_text: {type(cn_text)}")

# 2. 使用chat接口文字翻译成英文
response = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "translate from chinese to english: " + cn_text.text}, ], #str(cn_text)亦可
    temperature=0,
)
print(f"translate to english: {response}")

en_text = response.choices[0].message.content

# 3. 英文转换成语音
speech_file_path = "output_translate.mp3"
response = client.audio.speech.create(
    model="tts-1",
    voice="alloy",
    input=en_text,
    response_format="mp3",
    speed=1,
)
response.stream_to_file(speech_file_path)


origin voice: Transcription(text='你好,今天天气怎么样?')
type of cn_text: <class 'openai.types.audio.transcription.Transcription'>
translate to english: ChatCompletion(id='chatcmpl-9EvSr62Chn2ijLJR2WCps3CEtCg0r', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Hello, how's the weather today?", role='assistant', function_call=None, tool_calls=None))], created=1713344513, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_77a673219d', usage=CompletionUsage(completion_tokens=8, prompt_tokens=41, total_tokens=49))


/tmp/ipykernel_1019985/1032074807.py:44: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path)
